<a href="https://colab.research.google.com/github/pgmaximo/Imers-oIA_Alura/blob/main/Matza_%5BImers%C3%A3o_IA_Projeto%5D_Assistente_de_treinamento_GCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação
!pip install -U -q google-generativeai
!pip install -U -q pypdf2
!pip install -U -q gdown

In [ ]:
# Importações
from google.colab import auth
from google.colab import userdata
from google.colab import drive
from PyPDF2 import PdfReader
import os
import gdown
import numpy as np
import pandas as pd
import google.generativeai as genai

In [ ]:
# Configurando a chave API (Usando o Secret Keys do Google Colab)
genai.configure(api_key=userdata.get('api_key'))

# Definindo caminho da pasta com os PDFs
drive.mount('/content/drive')
pasta_pdf = '/content/drive/MyDrive/PDFs para IA'

# Função para extrair o texto completo do PDF e dividir em partes
def extrair_texto_pdf(caminho_arquivo, tamanho_parte=8000):
    with open(caminho_arquivo, 'rb') as f:
        reader = PdfReader(f)
        texto_completo = ""
        partes_texto = []
        for pagina in reader.pages:
            texto_completo += pagina.extract_text()
            if len(texto_completo) >= tamanho_parte:
                partes_texto.append(texto_completo[:tamanho_parte])
                texto_completo = texto_completo[tamanho_parte:]
        if texto_completo:
            partes_texto.append(texto_completo)
        return partes_texto

# Criando embeddings para cada PDF
embeddings = []
textos_completos = []
nomes_arquivos = []

for nome_arquivo in os.listdir(pasta_pdf):
    if nome_arquivo.endswith(".pdf"):
        caminho_arquivo = os.path.join(pasta_pdf, nome_arquivo)
        partes_texto = extrair_texto_pdf(caminho_arquivo)
        embeddings_arquivo = []
        texto_completo_arquivo = "" # Variável para armazenar o texto completo do PDF

        for parte_texto in partes_texto:
            texto_completo_arquivo += parte_texto # Concatena as partes do texto
            embedding = genai.embed_content(
                model="models/embedding-001",
                content=parte_texto,
                task_type="RETRIEVAL_DOCUMENT"
            )["embedding"]
            embeddings_arquivo.append(embedding)

        embedding_media = np.mean(embeddings_arquivo, axis=0)
        embeddings.append(embedding_media)

        # Adiciona os dados do PDF apenas uma vez
        textos_completos.append(texto_completo_arquivo)
        nomes_arquivos.append(nome_arquivo)

# Criando DataFrame com os dados dos PDFs
df_treinamentos = pd.DataFrame({
    "Nome do Arquivo": nomes_arquivos,
    "Texto Completo": textos_completos,
    "Embeddings": embeddings
})


# ----> Movendo configurações para fora do loop
# Configuração de geração
generation_config = {
    "temperature": 0.25,
    "top_p": 0.95,
    "top_k": 0,
}

# Configurações de segurança
safety_settings = [
    {"category": "hate", "threshold": "medium"},
    {"category": "harassment", "threshold": "medium"},
    {"category": "sexual", "threshold": "medium"},
    {"category": "dangerous", "threshold": "medium"},
]

# Configuração do modelo
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

# Criando o chat conversacional
chat = model.start_chat(history=[])

# Função para recomendar o treinamento mais adequado
def recomendar_treinamento(prompt_usuario):
    embedding_consulta = genai.embed_content(
        model="models/embedding-001",
        content=prompt_usuario,
        task_type="RETRIEVAL_QUERY"
    )["embedding"]

    # ----> Calculando similaridade após carregar o DataFrame
    similaridades = np.dot(np.stack(df_treinamentos["Embeddings"]), embedding_consulta)
    indice_treinamento = np.argmax(similaridades)

    nome_arquivo_treinamento = df_treinamentos.iloc[indice_treinamento]["Nome do Arquivo"]
    texto_treinamento = df_treinamentos.iloc[indice_treinamento]["Texto Completo"]

    # ----> Prompt mais conciso e focado
    resposta = chat.send_message(
        f"""
        ## Contexto:
        Você é um assistente de treinamento do Google Cloud Platform da Matza.
        Sua tarefa é ajudar os usuários a encontrar o treinamento ideal. Considere itens como Nivel de Experiencia, Topicos abordados e a sua descrição.

        ## Informações do Treinamento: {texto_treinamento}


        ## Exemplos 1:
        Seja conciso e claro, como neste exemplo:

        USUARIO: Quero aprender sobre Google Cloud. Sou totalmente iniciante

        Nome do Treinamento: Google Cloud Fundamentals: Core Infrastructure
        Descrição: Aborda os conceitos básicos do Google Cloud...
        Duração: 1 dia
        Formato: On Demand ou ILT
        Nível de Experiência: Iniciante
        Módulos: 9
        Laboratórios: 6
        Pré-Requisisto: Familiaridade com desenvolvimento de aplicativos
        Tópicos abordados: Compute Engine, Cloud Storage...
        Tópicos NÃO abordados: BigQuery, Borg e Colossus

        ## Exemplo 2:
        USUARIO: Quero aprender sobre Cloud Digital Leader

        Nome do Treinamento:Cloud Digital Leader
        Descrição:Aborda os conceitos básicos do Google Cloud...
        Duração:2 dias
        Formato:Instructor led (ILT)
        Nível de Experiência:Iniciante
        Módulos:9
        Laboratórios:6
        Pré-Requisisto:Familiaridade com desenvolvimento de aplicativos
        Tópicos abordados:Compute Engine, Cloud Storage...
        Tópicos NÃO abordados:BigQuery, Borg e Colossus

        ## Observações e considerações importante:
        ** Priorizar palavras em portugues
          *Introductory e Beginner é a mesma coisa que Iniciante.
          *Intermediate é a mesma coisa que intermediario
          *Advanced é a mesma coisa que avançado

        ## Tarefa:
        Extraia as seguintes informações do treinamento:
        * Nome do Treinamento
        * Faça uma pequena descrição detalhada, relacionando o conhecimento que o treinamento traz, o Nivel de experiencia minima e os pre requisistos
        * Duração
        * Formato
        * Nível de Experiência
        * Módulos
        * Pré-requisito (Quero que voce anSalise para ver se o Pré-requisito dito no arquivo tem haver com o que é ensinado ou se é verdade)
        * Laboratórios
        * Tópicos abordados
        * Tópicos NÃO abordados
        """
    )
    return resposta.text, nome_arquivo_treinamento

# ----> Feedback do Usuário (requer implementação adicional)
def registrar_feedback(prompt_usuario, nome_arquivo_treinamento, feedback):
    # Implemente aqui a lógica para armazenar o feedback do usuário.
    # Exemplo: salvar em um arquivo CSV ou banco de dados.
    print(f"Feedback registrado: {feedback} para o prompt '{prompt_usuario}' e treinamento '{nome_arquivo_treinamento}'")

# Loop principal do chatbot
prompt_usuario = input("Descreva suas necessidades de treinamento: ")
while prompt_usuario != "Desligar":
    recomendacao, nome_arquivo = recomendar_treinamento(prompt_usuario)
    print(recomendacao)
    print(f"Você pode encontrar mais informações no arquivo: {nome_arquivo}")

    feedback = input("A recomendação foi útil? (s/n): ").lower()
    registrar_feedback(prompt_usuario, nome_arquivo, feedback)

    prompt_usuario = input("=> ")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
